In [12]:
pip install langchain

In [13]:
pip install yandex_chain

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install langchain_text_splitters

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install Chroma

Note: you may need to restart the kernel to use updated packages.


In [16]:
from langchain.embeddings import HuggingFaceEmbeddings

In [17]:
pip install ragas

Note: you may need to restart the kernel to use updated packages.


In [18]:
import argparse
import os
import shutil
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma
from langchain.document_loaders import DirectoryLoader
import argparse
from langchain.vectorstores.chroma import Chroma
import langchain
from langchain.prompts import ChatPromptTemplate
import langchain.chains
import langchain.prompts
from yandex_chain import YandexLLM, YandexEmbeddings
CHROMA_PATH = "chroma"
DATA_PATH = "dataa"
def load_documents():
    document_loader =  DirectoryLoader(DATA_PATH, glob="*.txt", show_progress=True, recursive=True)
    return document_loader.load()
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=600,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)
def add_to_chroma(chunks: list[Document]):
    # Load the existing database.
    db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=YandexEmbeddings(config="config.json")
    )

    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)
    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")

def calculate_chunk_ids(chunks):

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        chunk.metadata["id"] = chunk_id

    return chunks

def clear_database():
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)


In [19]:
documents = load_documents()
chunks = split_documents(documents)
add_to_chroma(chunks)

 85%|█████████████████████████████████████████████████████████████████████▉            | 52/61 [00:05<00:00, 10.10it/s]

Number of existing documents in DB: 869
✅ No new documents to add


In [20]:
import argparse
from langchain.vectorstores.chroma import Chroma
import langchain
from langchain.prompts import ChatPromptTemplate
import langchain.chains
import langchain.prompts
from yandex_chain import YandexLLM, YandexEmbeddings

PROMPT_TEMPLATE = """
Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста. Тебе очень нравится учиться в МАИ. Ты ассистент студента Московского Авиационного Института
{context}

---

Ответьте на вопрос, основываясь на приведенном выше контексте: {question}
"""
def query_rag(query_text: str):
    # Prepare the DB.
    embedding_function=YandexEmbeddings(config="config.json")
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    model = YandexLLM(config="config.json")
    response_text = model.invoke(prompt)
    print(prompt)
    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text

In [29]:
# Запрос, который вы хотите отправить
query_text = "Зачем люди поступают в магистратуру?"
# Вызов функции с запросом
response = query_rag(query_text)
# Вывод ответа
print(response)

Human: 
Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста. Тебе очень нравится учиться в МАИ. Избегай слов, выражающие неуверенность.
магистратура

20.03.01

Междисциплинарный экзамен

Специализированное

Материаловедение и технология                                   Бюджетная, в соответствии с программой

22.04.01   высшее образование

материалов                                                      платная    бакалавриата по направлению

магистратура

22.03.01

Междисциплинарный экзамен

Специализированное

Бюджетная, в соответствии с программой

Металлургия                       22.04.02   высшее образование

---

2021/22 учебный год 3. В какое время проводятся занятия в магистратуре (очная): утром или вечером? Сколько раз в неделю? Удаётся ли студентам совмещать обучение в очной магистратуре на экономике и работу? 4. Где студенты экономического направления проходят стажировку? Насколько успешно студенты находят работу после окончания МАИ в 

In [30]:
# тестирование RAG
EVAL_PROMPT = """
Expected Response: {expected_response}
Actual Response: {actual_response}
---
(Answer with 'true' or 'false') Does the actual response match the expected response? 
"""


def test_monopoly_rules():
    assert query_and_validate(
        question="Кто является директором 8 института",
        expected_response="«Директором является Крылов Сергей Сергеевич»",
    )


def test_ticket_to_ride_rules():
    assert query_and_validate(
        question="Документы, необходимые для поступления в аспирантуру",
        expected_response="""Документы, необходимые для поступления в аспирантуру:
Заявление о приеме на обучение по программам подготовки научных и научно-педагогических кадров в аспирантуре (принимается только в печатной форме).
Документ (документы), удостоверяющий личность и гражданство поступающего (ксерокопия страниц паспорта 1,2,3.4.5).
Копия диплома специалиста или диплома магистра с приложением (поступающий может при подаче заявления о приеме не представлять документ установленного образца; при этом поступающий дает обязательство представить указанный документ не позднее 10 июля 2024г.).
Анкета .
2 фотографии поступающего, размер 4х6. 
Копия удостоверения гражданина, подлежащего призыву на военную службу, приписное свидетельство или военный билет (все страницы).
Реферат по избранной научной специальности. Бланк отзыва предполагаемого научного руководителя.""",
    )


def query_and_validate(question: str, expected_response: str):
    response_text = query_rag(question)
    prompt = EVAL_PROMPT.format(
        expected_response=expected_response, actual_response=response_text
    )

    model = YandexLLM(config="config.json")
    evaluation_results_str = model.invoke(prompt)
    evaluation_results_str_cleaned = evaluation_results_str.strip().lower()

    print(prompt)

    if "true" in evaluation_results_str_cleaned:
        # Print response in Green if it is correct.
        print("\033[92m" + f"Response: {evaluation_results_str_cleaned}" + "\033[0m")
        return True
    elif "false" in evaluation_results_str_cleaned:
        # Print response in Red if it is incorrect.
        print("\033[91m" + f"Response: {evaluation_results_str_cleaned}" + "\033[0m")
        return False
    else:
        raise ValueError(
            f"Invalid evaluation result. Cannot determine if 'true' or 'false'."
        )

In [23]:
test_monopoly_rules()
test_ticket_to_ride_rules()

Human: 
Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста. Тебе очень нравится учиться в МАИ. Избегай слов, выражающие неуверенность.
Дирекция 8го института: Крылов Сергей Сергеевич директор института 8, каб. 607, телефон +7 499 158-09-06, почта krylov@mai.ru Кучева Наталья Александровна заместитель директора по учебной работе 8 института, каб. 607, телефон +7 499 158-16-77, почта kuchevana@mai.ru Северина Наталья Сергеевна заместитель директора по научной работе 8 института, начальник НИО-8, каб. 607, телефон +7 499 158-47-67, почта severinans@mai.ru Селиванова Ольга Сергеевна и. о. заместителя директора по внеучебной и воспитательной

---

Дирекция 8го института: Крылов Сергей Сергеевич директор института 8, каб. 607, телефон +7 499 158-09-06, почта krylov@mai.ru Кучева Наталья Александровна заместитель директора по учебной работе 8 института, каб. 607, телефон +7 499 158-16-77, почта kuchevana@mai.ru Северина Наталья Сергеевна заместитель ди